In [1]:
import numpy as np
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10
from torch.optim import SGD
from torch.nn import CrossEntropyLoss, MSELoss
from pennylane import NesterovMomentumOptimizer
from thesis.qcnn import QCNN

dims = (16, 16), (28, 28), (32, 32)
num_trials = 1

In [2]:
from thesis.operation.ansatz import ConvolutionAnsatz
from thesis.ml.data import Data
from thesis.ml.optimize import Optimizer
from thesis.ml.data import image_transform
from thesis.logger import Logger

data = Data(FashionMNIST, image_transform((16, 16)), None, classes=[0, 1], batch_size=(80, 1000))
optimizer = Optimizer(SGD, lr=0.01, momentum=0.9, nesterov=True)
qcnn = QCNN(data, optimizer, CrossEntropyLoss(), Logger.from_schema([("a",int)], name="potato"))

for _ in range(num_trials):
    accuracy = qcnn(ConvolutionAnsatz, (16, 16))
    print(f"New ansatz: {accuracy=:.3%}")

INFO:potato:Accuracy: 93.050%


New ansatz: accuracy=93.050%


In [3]:
from thesis.logger import Logger

test = Logger.from_schema([("a", int)], name="yeetus")

In [4]:
test.log(3)

INFO:yeetus:{'a': 3}
